# Basic func

In [2]:
import tensorflow as tf
import numpy as np

In [5]:
node1 = tf.constant(3)
sess = tf.Session()
print (sess.run([node1]))
print (sess.run(node1))==3

[3]
3


False

In [7]:
node2 = tf.constant(2)
node3 = tf.add(node1,node2)
print("node3:",sess.run(node3))

node3: 5


In [14]:
a = tf.placeholder(tf.float32)
b= tf.placeholder(tf.float32)
adder_node = a + b

In [15]:
print(sess.run(adder_node,{a:[1,3],b:[2,4]}))

[ 3.  7.]


In [16]:
add_triple = adder_node*3
print(sess.run(add_triple,{a:[1,3],b:[2,4]}))

[  9.  21.]


In [17]:
W = tf.Variable([.3],tf.float32)
b = tf.Variable([-.3],tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [18]:
init = tf.global_variables_initializer()
sess.run(init)

In [19]:
print (sess.run(linear_model,{x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [22]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss,{x:[1,2,3,4],y:[-1,-1,-2,-3]}))

24.66


### Assign 赋值操作

In [30]:
fixW = tf.assign(W,[-1.20])
fixb = tf.assign(b,[1.])
print(sess.run(fixW))
#sess.run([fixW,fixb])
print(sess.run(loss,{x:[1,2,3,4],y:[-1,-1,-2,-3]}))


[-1.20000005]
1.8


## tf.train API

In [31]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [34]:
sess.run(init)

#squared_deltas = tf.square(linear_model - y)
#loss = tf.reduce_sum(squared_deltas)

for i in range(1000):
    sess.run(train,{x:[1,2,3,4], y:[0,-1,-2,-3]})
    if i%100==0:
        print(sess.run(W),sess.run(b))
print(sess.run([W,b]))

[-0.21999997] [-0.456]
[-0.84270465] [ 0.53753263]
[-0.95284992] [ 0.86137295]
[-0.98586655] [ 0.95844597]
[-0.99576342] [ 0.98754394]
[-0.99873012] [ 0.99626648]
[-0.99961936] [ 0.99888098]
[-0.99988592] [ 0.9996646]
[-0.99996579] [ 0.99989945]
[-0.99998969] [ 0.99996972]
[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# A complete program

In [35]:
#import tensorflow
#import numpy

#init wb
W = tf.Variable([1.3],tf.float32)
b = tf.Variable([-.3],tf.float32)
#input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
#loss
loss = tf.reduce_sum(tf.square(linear_model - y))
#optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train  = optimizer.minimize(loss)
x_train = [1,3,4,5]
y_train = [0,2,3,4]

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train,{x:x_train,y:y_train})
    
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
    

W: [ 0.99999946] b: [-0.99999779] loss: 3.52429e-12


In [37]:
features = [tf.contrib.layers.real_valued_column("x",dimension = 1)]
estimator = tf.contrib.learn.LinearRegressor(feature_columns = features)
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn  =tf.contrib.learn.io.numpy_input_fn({"x":x},y,batch_size=4,num_epochs=1000)
estimator.fit(input_fn=input_fn,steps = 1000)
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000A368748>, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_task_id': 0, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_environment': 'local', '_save_checkpoints_secs': 600, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\sky\App

{'global_step': 1000, 'loss': 2.4049285e-08}

In [38]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000BAE55C0>, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_task_id': 0, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_environment': 'local', '_save_checkpoints_secs': 600, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\sky\AppData\Local\Temp\tmpbshucjmo\model.ckpt.
INFO:tensorflow:step = 1, loss = 17.2307481665
INFO:tensorflow:global_step/sec: 1562.5
INFO:tensorflow:step = 101, loss = 0.0292244105427
INFO:tensorflow:global_step/sec: 1562.5
INFO:tensorflow:step = 201, loss = 0.00181773688304
INFO:tensorflow:global_step/sec: 1694.91
INFO:tensorflow:st